In [10]:
%pip install kiwipiepy
%pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from kiwipiepy import Kiwi
from transformers import BertTokenizerFast ,BertForMaskedLM,FillMaskPipeline
import torch

In [2]:
kiwi = Kiwi()
text = "1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배되었다."

# 형태소 분석 및 품사 태깅
result = kiwi.tokenize(text)

# KoBERT 토크나이저 로드
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

# Kiwi 형태소 분석 결과를 문자열로 변환
kiwi_tokens = [token.form for token in result]
kiwi_text = ' '.join(kiwi_tokens)

kiwi_tokens_with_pos = [f"{token.form}_{token.tag}" for token in result]
kiwi_text_with_pos = ' '.join(kiwi_tokens_with_pos)

print("Kiwi Tokens with POS:", kiwi_tokens_with_pos)
# KoBERT 토크나이저로 토크나이징
kobert_tokens = tokenizer.tokenize(kiwi_text)
kobert_input_ids = tokenizer.convert_tokens_to_ids(kobert_tokens)
only_kobert_tokens = tokenizer.tokenize(text)
kobert_tokens_pos = tokenizer.tokenize(kiwi_text_with_pos)
kobert_input_ids_pos = tokenizer.convert_tokens_to_ids(kobert_tokens_pos)
# 결과 출력

print("Kiwi Tokens:", kiwi_tokens)
print("KoBERT Tokens:", kobert_tokens)
print("KoBERT Input IDs:", kobert_input_ids)
print("token Num:",len(kobert_input_ids))

print("only_kobert_tokens:",only_kobert_tokens)

# 결과 출력
print("KoBERT Tokens_pos:", kobert_tokens_pos)
print("KoBERT Input IDs_pos:", kobert_input_ids_pos)
print("token Num:",len(kobert_input_ids_pos))

Kiwi Tokens with POS: ['1989_SN', '년_NNB', '2_SN', '월_NNB', '15_SN', '일_NNB', '여의도_NNP', '농민_NNG', '폭력_NNG', '시위_NNG', '를_JKO', '주도_NNG', '하_XSV', 'ᆫ_ETM', '혐의_NNG', '로_JKB', '지명_NNG', '수배_NNG', '되_XSV', '었_EP', '다_EF', '._SF']
Kiwi Tokens: ['1989', '년', '2', '월', '15', '일', '여의도', '농민', '폭력', '시위', '를', '주도', '하', 'ᆫ', '혐의', '로', '지명', '수배', '되', '었', '다', '.']
KoBERT Tokens: ['1989', '년', '2', '월', '15', '일', '여의도', '농민', '폭력', '시위', '를', '주도', '하', '[UNK]', '혐의', '로', '지명', '수배', '되', '었', '다', '.']
KoBERT Input IDs: [9190, 761, 22, 1479, 3749, 1507, 7915, 6290, 5206, 5754, 1022, 4682, 1889, 1, 4456, 991, 7119, 18627, 859, 1417, 809, 18]
token Num: 22
only_kobert_tokens: ['1989', '##년', '2', '##월', '15', '##일', '여의도', '농민', '폭력', '시위', '##를', '주도', '##한', '혐의', '##로', '지명', '##수', '##배', '##되', '##었', '##다', '.']
KoBERT Tokens_pos: ['1989', '_', 'SN', '년', '_', 'N', '##N', '##B', '2', '_', 'SN', '월', '_', 'N', '##N', '##B', '15', '_', 'SN', '일', '_', 'N', '##N', '##B', '여의도', '_', '

In [13]:
import json

with open("./6_modern_data.json", 'rb') as f :
    data = json.load(f)

sentence_token = []


for info in enumerate(data.values()): 
    form = info[1]["sentence"]
    form = form.replace(info[1]["aux_tokens"][0]["form"], "[MASK]")
    sentence_token.append(form)

In [53]:
print(len(sentence_token))
print(len(data.values()))

13279
13279


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')
model = BertForMaskedLM.from_pretrained('klue/bert-base')

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
model.config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.34.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [24]:
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [29]:
pip(sentence_token[0])

[{'score': 0.1582077294588089,
  'token': 28262,
  'token_str': '산더미',
  'sequence': '담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 산더미 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.'},
 {'score': 0.06853462755680084,
  'token': 28000,
  'token_str': '깨알',
  'sequence': '담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 깨알 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.'},
 {'score': 0.055071957409381866,
  'token': 809,
  'token_str': '다',
  'sequence': '담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 다 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.'},
 {'score': 0.05441555753350258,
  'token': 24083,
  'token_str': '거미줄',
  'sequence': '담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 거미줄 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.'},
 {'score': 0.037816449999809265,
  'token': 3657,
  'token_str':

In [61]:
from tqdm import tqdm

# 가정: `model`과 `tokenizer`는 이미 정의되어 있음
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

Similar_data = {"data": []}
dict = {"data": [{"para": [{"label": [{}], "context": ""}]}]}

# tqdm을 사용하여 zip을 감싸줍니다.
for sentence, raw_context in tqdm(zip(sentence_token, data.values()), total=len(sentence_token)):
    labels = {"label": []}
    para = {"para": []}
    len_token = tokenizer.tokenize(sentence)

    # 입력 길이 초과 처리
    if len(len_token) >= 512:
        len_token = len_token[:500]

    context = tokenizer.convert_tokens_to_string(len_token)

    labels["context"] = raw_context["sentence"]
    labels["label"].extend(pip(context))

    para["para"].append(labels)
    Similar_data["data"].append(para)


100%|██████████| 13279/13279 [11:57<00:00, 18.50it/s]


In [62]:
with open('./Similar_data.json', 'w', encoding='utf-8') as file :
    json.dump(Similar_data, file, indent='\t', ensure_ascii=False)